<a href="https://colab.research.google.com/github/sugarforever/LlamaIndex-Tutorials/blob/main/LlamaPack_Beginners_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install llama-index pypdf -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.3 MB/s eta 0:00:00


# What is LlamaPack?

Llama Packs are a community-driven hub of prepackaged modules/templates - [llama_packs](https://llamahub.ai/?tab=llama_packs)

# How to use LlamaPack

There are 2 ways to use it.

## CLI

```shell
llamaindex-cli download-llamapack <pack_name> --download-dir <pack_directory>
```

## Python

```python
from llama_index.llama_pack import download_llama_pack
pack_cls = download_llama_pack("<pack_name>", "<pack_directory>")
```

# Use Cases

In this guide, we will see 4 useful Llama Packs that boost RAG pipeline development.

## Prepare Document

We will use the PDF document [Analysis and Comparison between
Optimism and StarkNet](https://arxiv.org/pdf/2210.16610.pdf) for demo purpose.

In [9]:
!wget -O Analysis_and_Comparison_between_Optimism_and_StarkNet.pdf https://ceur-ws.org/Vol-3460/papers/DLT_2023_paper_16.pdf

--2024-01-30 21:20:29--  https://ceur-ws.org/Vol-3460/papers/DLT_2023_paper_16.pdf
Resolving ceur-ws.org (ceur-ws.org)... 137.226.34.231
Connecting to ceur-ws.org (ceur-ws.org)|137.226.34.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1031515 (1007K) [application/pdf]
Saving to: ‘Analysis_and_Comparison_between_Optimism_and_StarkNet.pdf’

Analysis_and_Compar 100%[===================>]   1007K  1.91MB/s    in 0.5s    

2024-01-30 21:20:30 (1.91 MB/s) - ‘Analysis_and_Comparison_between_Optimism_and_StarkNet.pdf’ saved [1031515/1031515]



In [10]:
from llama_index.readers import PDFReader

reader = PDFReader()
documents = reader.load_data("Analysis_and_Comparison_between_Optimism_and_StarkNet.pdf")

In [11]:
len(documents)

14

In [13]:
from llama_index.node_parser import SimpleNodeParser
nodes = SimpleNodeParser.from_defaults().get_nodes_from_documents(documents)

In [15]:
len(nodes)

14

In [17]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [14]:
from llama_index.llama_pack import download_llama_pack

In [22]:
import nest_asyncio
nest_asyncio.apply()

In [35]:
QUESTION = "What is the goal of validity rollup?"

## HybridFusionRetrieverPack

Vector retriever and BM25 retriever

In [18]:
llama_pack = download_llama_pack(
    "HybridFusionRetrieverPack", "./hybrid_fusion_pack"
)

hybrid_fusion_pack = llama_pack(
    nodes,
    chunk_size=256,
    vector_similarity_top_k=2,
    bm25_similarity_top_k=2
)

In [55]:
response = hybrid_fusion_pack.run(QUESTION)

Generated queries:
1. How does validity rollup contribute to data analysis?
2. What are the benefits of implementing validity rollup in data management?
3. What are the best practices for ensuring the accuracy of validity rollup results?


In [56]:
response

Response(response='The goal of Validity Rollups is to ensure the validity of transaction execution at light nodes using cryptographic methods, specifically validity proofs.', source_nodes=[NodeWithScore(node=TextNode(id_='4a33a79a-8b1c-449d-bdb5-53b9988fc32b', embedding=None, metadata={'page_label': '1', 'file_name': 'Analysis_and_Comparison_between_Optimism_and_StarkNet.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1306ab05-89ca-4bde-bd78-84cf68d035ef', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'Analysis_and_Comparison_between_Optimism_and_StarkNet.pdf'}, hash='6a6dca3021dbcf3d9b8b972c7bece6be00571a44bba020db0a3133da292aa168'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2ba70a1c-681d-4da3-8565-bf155b47abdf', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='10c0064ccdfb2aae42f13e29eefc313b8ae40bafe46dbe91192710c1f9c5025c')}, text='Optimi

In [40]:
response.response

'The goal of Validity Rollups is to ensure the validity of transaction execution at light nodes using cryptographic methods, specifically validity proofs.'

## QueryRewritingRetrieverPack

This pack generates multiple queries based on the user's query.

In [57]:
llama_pack = download_llama_pack(
    "QueryRewritingRetrieverPack", "./query_rewriting_pack"
)

query_rewriting_pack = llama_pack(
    nodes,
    chunk_size=256,
    vector_similarity_top_k=2,
)

In [37]:
response = query_rewriting_pack.run(QUESTION)

Generated queries:
1. How does validity rollup improve data accuracy?
2. What are the benefits of implementing validity rollup in data analysis?
3. What are the best practices for conducting validity rollup in data validation processes?


In [38]:
response.response

'The goal of Validity Rollups is to address the problem of scalability in decentralized blockchains by analyzing the trade-off between transaction throughput and hardware requirements to run a node. Validity Rollups aim to provide on-chain verification of blocks executed off-chain, using fault or validity proofs.'

## SentenceWindowRetrieverPack

 `SentenceWindowRetrieverPack` loads a document, chunks it up, adds surrounding context as metadata to each chunk, and during retrieval inserts the context back into each chunk for response synthesis.

In [41]:
llama_pack = download_llama_pack(
    "SentenceWindowRetrieverPack", "./sentence_window_retriever_pack"
)

# Pass documents to build the pack instance
sentence_window_retriever_pack = llama_pack(documents)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1332: RuntimeWarning: coroutine 'BaseRetriever.aretrieve' was never awaited
  _C._initExtension(manager_path())
/usr/local/lib/python3.10/dist-packages/torch/__init__.py:1332: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  _C._initExtension(manager_path())
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [42]:
response = sentence_window_retriever_pack.run(QUESTION)

In [43]:
response.response

'The goal of validity rollup is to cryptographically prove the validity of the state transition given the sequence of transactions with a short proof that can be verified sub-linearly compared to the time of the original computations.'

In [44]:
response

Response(response='The goal of validity rollup is to cryptographically prove the validity of the state transition given the sequence of transactions with a short proof that can be verified sub-linearly compared to the time of the original computations.', source_nodes=[NodeWithScore(node=TextNode(id_='0549cd82-9c88-420b-af9a-b22aef682ca2', embedding=None, metadata={'window': 'The oracle\nis implemented by minigeth and replaces the database.  Queries are made to other nodes to\nobtain the preimages.\n 3.  Validity Rollups\nThe goal of a Validity Rollup is to cryptographically prove the validity of the state transition\ngiven the sequence of transactions with a short proof that can be verified sub-linearly compared\nto the time of the original computations.\n These kind of certificates are called computational integrity proofs and are practically imple-\nmented with SNARKs (Succint Non-interactive ARgument of Knowledge), which use arithmetic\ncircuits as their computational model.  Differ

## RecursiveRetrieverSmallToBigPack

`RecursiveRetrieverSmallToBigPack` loads a document, builds a hierarchical node graph (with bigger parent nodes and smaller child nodes)

In [50]:
llama_pack = download_llama_pack(
    "RecursiveRetrieverSmallToBigPack", "./recursive_retriever_stb_pack"
)

recursive_retriever_stb_pack = llama_pack(documents)


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [53]:
response = recursive_retriever_stb_pack.run(QUESTION)

In [54]:
response.response

'The goal of a Validity Rollup is to cryptographically prove the validity of the state transition given the sequence of transactions with a short proof that can be verified sub-linearly compared to the time of the original computations.'

## Use the Modules in the LLamaPack Separately

We are able to fetch the modules included in the pack and use them separately.

In [58]:
# get the sentence vector index
index = sentence_window_retriever_pack.sentence_index

# get the node parser
node_parser = sentence_window_retriever_pack.node_parser

# get the metadata replacement postprocessor
postprocessor = sentence_window_retriever_pack.postprocessor

# get the query engine
query_engine = sentence_window_retriever_pack.query_engine

In [46]:
index

In [47]:
node_parser

SentenceWindowNodeParser(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ba4623e6560>, id_func=<function default_id_func at 0x7ba469323130>, sentence_splitter=<function split_by_sentence_tokenizer.<locals>.split at 0x7ba461ae5f30>, window_size=3, window_metadata_key='window', original_text_metadata_key='original_text')

In [48]:
postprocessor

MetadataReplacementPostProcessor(callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7ba46384f730>, target_metadata_key='window')

In [49]:
query_engine